0 导入包

In [1]:
# 加载数据和数据处理
from datasets import load_dataset, DatasetDict
from datasets import Audio

# 加载模型
from transformers import AutoModelForAudioClassification
from transformers import AutoFeatureExtractor

# 训练
from transformers import Trainer
from transformers import TrainingArguments

# 评估指标
import evaluate
import numpy as np

# 计算资源
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16


model_id = "ntu-spml/distilhubert"

1 加载数据集

In [2]:
gtzan = load_dataset("marsyas/gtzan", "all") # 1000个30s歌曲片段，分10个类别的数据集
gtzan = gtzan["train"].train_test_split(train_size=0.9,test_size=0.1,seed=42, shuffle=True, ) # 切分数据集

# 调整采样率
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)
sampling_rate = feature_extractor.sampling_rate
gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

gtzan = gtzan.rename_column("genre", "label") # 重命名
gtzan

d:\env_config\py\lib\site-packages\datasets\load.py:1491: FutureWarning: The repository for marsyas/gtzan contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/marsyas/gtzan
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'label'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'label'],
        num_rows: 100
    })
})

2 加载模型

In [3]:
# 构建标签，序号对应正反对应字典
a = gtzan["train"].features["label"].names # 标签列表
a

['blues',
 'classical',
 'country',
 'disco',
 'hiphop',
 'jazz',
 'metal',
 'pop',
 'reggae',
 'rock']

In [4]:
id_label_dict = {str(i):a[i]
                 for i in range(len(a))
                 }
id_label_dict

{'0': 'blues',
 '1': 'classical',
 '2': 'country',
 '3': 'disco',
 '4': 'hiphop',
 '5': 'jazz',
 '6': 'metal',
 '7': 'pop',
 '8': 'reggae',
 '9': 'rock'}

In [5]:
label_id_dict ={a[i]:str(i)
                for i in range(len(a))
                }
label_id_dict

{'blues': '0',
 'classical': '1',
 'country': '2',
 'disco': '3',
 'hiphop': '4',
 'jazz': '5',
 'metal': '6',
 'pop': '7',
 'reggae': '8',
 'rock': '9'}

In [6]:
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=len(a),
    label2id=label_id_dict,
    id2label=id_label_dict,
)


Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


3 预处理-feature+留下小于30s的数据

In [7]:
gtzan = gtzan.map(
    lambda x: 
    feature_extractor(
        [i["array"] for i in x["audio"]], # 要处理的数据，
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * 30), # 最长为30s音频
        truncation=True,
        return_attention_mask=True,
    ),
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1, # 进程数量
)
gtzan

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['label', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

In [8]:
type(gtzan['train'][0]['label'])

int

4 定义评估指标

In [9]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

5 数据预处理器

In [10]:
# 在数据传进模型之前再做一遍数据处理
# 因为实际运行中会因为数据类型错误而报错，所以加了这一层预处理
from transformers import DataCollatorWithPadding
import torch

class DataCollatorWithLabelConversion(DataCollatorWithPadding):
    def __call__(self, features):
        batch = super().__call__(features)
        if "labels" in batch:
            batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)  # 转换为 LongTensor
        return batch

data_collator = DataCollatorWithLabelConversion(tokenizer=feature_extractor)


6 训练参数

In [14]:
trainer = Trainer(
    model = model,
    train_dataset=gtzan["train"],
    eval_dataset=gtzan["test"],

    tokenizer=feature_extractor,

    compute_metrics=compute_metrics, # 训练后评估

    data_collator=data_collator,

    args=TrainingArguments(
        # ——————模型保存——————
        output_dir="fted-gtzan",  # 模型保存路径
        save_strategy="epoch",

        # ——————计算量——————
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2,
        num_train_epochs=10,

        # ——————训练精度——————
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),

        # ——————更新参数——————
        learning_rate=5e-5,
        warmup_ratio=0.1,

        # ——————验证参数——————
        evaluation_strategy="epoch", # 按照周期评估
        per_device_eval_batch_size=4,

        # ——————log参数——————
        logging_steps=5,

        # ——————训练后评估——————
        metric_for_best_model="accuracy",

    )
)

d:\env_config\py\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


7 开始训练

In [15]:
trainer.train()

  0%|          | 0/1120 [00:00<?, ?it/s]

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3680\1108811776.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)  # 转换为 LongTensor


{'loss': 0.1668, 'grad_norm': 2.1779890060424805, 'learning_rate': 2.2321428571428573e-06, 'epoch': 0.04}
{'loss': 0.3099, 'grad_norm': 5.167806625366211, 'learning_rate': 4.464285714285715e-06, 'epoch': 0.09}
{'loss': 0.2577, 'grad_norm': 5.803153991699219, 'learning_rate': 6.696428571428572e-06, 'epoch': 0.13}
{'loss': 0.276, 'grad_norm': 11.999265670776367, 'learning_rate': 8.92857142857143e-06, 'epoch': 0.18}
{'loss': 0.2078, 'grad_norm': 8.021317481994629, 'learning_rate': 1.1160714285714287e-05, 'epoch': 0.22}
{'loss': 0.4447, 'grad_norm': 14.069929122924805, 'learning_rate': 1.3392857142857144e-05, 'epoch': 0.27}
{'loss': 0.2241, 'grad_norm': 1.6829771995544434, 'learning_rate': 1.5625e-05, 'epoch': 0.31}
{'loss': 0.2214, 'grad_norm': 7.3764238357543945, 'learning_rate': 1.785714285714286e-05, 'epoch': 0.36}
{'loss': 0.3646, 'grad_norm': 7.204859256744385, 'learning_rate': 2.0089285714285717e-05, 'epoch': 0.4}
{'loss': 0.2672, 'grad_norm': 14.202727317810059, 'learning_rate': 2.

  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 0.6210741996765137, 'eval_accuracy': 0.79, 'eval_runtime': 20.5157, 'eval_samples_per_second': 4.874, 'eval_steps_per_second': 1.219, 'epoch': 1.0}


...

  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 0.6968475580215454, 'eval_accuracy': 0.8, 'eval_runtime': 20.4855, 'eval_samples_per_second': 4.882, 'eval_steps_per_second': 1.22, 'epoch': 2.0}


C:\Users\Administrator\AppData\Local\Temp\ipykernel_3680\1108811776.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)  # 转换为 LongTensor


{'loss': 0.1729, 'grad_norm': 1.4056661128997803, 'learning_rate': 4.41468253968254e-05, 'epoch': 2.04}
{'loss': 0.1627, 'grad_norm': 7.254873275756836, 'learning_rate': 4.3898809523809523e-05, 'epoch': 2.09}


: 